# GeoSpatial Data Project

In [1]:
# Librerias importadas token
import os
from dotenv import load_dotenv

In [2]:
# Librerias
import requests
import json
import pandas as pd

In [3]:
# Librerias visualizacion json
import pyjsonviewer

In [4]:
load_dotenv()

True

### asignamos los token de la carpeta .env a unas varibles que creamos

In [5]:
token = os.getenv("Client_Id")
token2 = os.getenv("Client_Secret")

#### con geocode buscamos la longitud y la latitud de la direccion que le demos

In [6]:
donde = "Plaza de Colón, Madrid"

##### funcion para obtener la dirección (latitud y longitud de una direccion que le pasemos)

In [7]:
def geocode(direccion):
    """
    Esta función saca las coordenadas de la dirección que le pases
    """
    data = requests.get(f"https://geocode.xyz/{direccion}?json=1").json()
    try:
        return {
            "type": "Point",
            "coordinates": [data["latt"], data["longt"]]
        }
    
    except:
        return data

In [8]:
donde = "Plaza de Colón, Madrid"
donde1 = "Heron Diversia, Avenida de Bruselas, Alcobendas"
donde2 = "Heron City Las Rozas, Calle Camilo José Cela, Las Rozas de Madrid"

In [9]:
madrid = geocode(donde)

In [10]:
url_query = 'https://api.foursquare.com/v2/venues/explore'

In [11]:
parametros = {
    "client_id": token,
    "client_secret": token2,
    "v": "20180323",
    "ll": f"{madrid['coordinates'][0]}, {madrid['coordinates'][1]}",
    "query": "Ciudad_Madrid",
    "limit": 200   
}

In [12]:
resp_madrid = requests.get(url_query, params=parametros).json()

In [13]:
# resp_madrid

In [14]:
alcobendas = geocode(donde1)

In [15]:
parametros = {
    "client_id": token,
    "client_secret": token2,
    "v": "20180323",
    "ll": f"{alcobendas['coordinates'][0]}, {alcobendas['coordinates'][1]}",
    "query": "Ciudad_Alcobendas",
    "limit": 200   
}

In [16]:
resp_alcobendas = requests.get(url_query, params=parametros).json()

In [17]:
# resp_alcobendas

In [18]:
rozas = geocode(donde2)

In [19]:
parametros = {
    "client_id": token,
    "client_secret": token2,
    "v": "20180323",
    "ll": f"{rozas['coordinates'][0]}, {rozas['coordinates'][1]}",
    "query": "Ciudad_Rozas",
    "limit": 200   
}

In [20]:
resp_rozas = requests.get(url_query, params=parametros).json()

In [21]:
datos_finales = resp_rozas["response"]["groups"][0]["items"]

In [22]:
datos_finales[1]["venue"]

{'id': '4b02fe9df964a520b94b22e3',
 'name': 'Las Rozas Village',
 'contact': {},
 'location': {'address': 'C/ Juan Ramon Jimenez 3',
  'lat': 40.5184364135419,
  'lng': -3.900519847869873,
  'labeledLatLngs': [{'label': 'display',
    'lat': 40.5184364135419,
    'lng': -3.900519847869873}],
  'distance': 201,
  'postalCode': '28232',
  'cc': 'ES',
  'city': 'Las Rozas',
  'state': 'Madrid',
  'country': 'España',
  'formattedAddress': ['C/ Juan Ramon Jimenez 3',
   '28232 Las Rozas Madrid',
   'España']},
 'categories': [{'id': '5744ccdfe4b0c0459246b4df',
   'name': 'Outlet Mall',
   'pluralName': 'Outlet Malls',
   'shortName': 'Outlet Mall',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/mall_',
    'suffix': '.png'},
   'primary': True}],
 'verified': True,
 'stats': {'tipCount': 0,
  'usersCount': 0,
  'checkinsCount': 0,
  'visitsCount': 0},
 'beenHere': {'count': 0,
  'lastCheckinExpiredAt': 0,
  'marked': False,
  'unconfirmedCount': 0},
 'photos': {'count'

In [23]:
from functools import reduce
import operator

In [24]:
mapa_nombre = ["venue", "name"]
mapa_latitud = ["venue", "location", "lat"]
mapa_longitud = ["venue", "location", "lng"]

In [25]:
def getFromDict(diccionario, mapa):
    return reduce(operator.getitem,mapa,diccionario)

In [26]:
def type_point(lista):
    return {"type": "Point", 
            "coordinates": lista }

In [27]:
unjsonnuevo = []
for dic in datos_finales:
    paralista= {}
    paralista["nombre"]= getFromDict(dic,mapa_nombre)
    paralista["latitud"]= getFromDict(dic,mapa_latitud)
    paralista["longitud"]= getFromDict(dic,mapa_longitud)
    paralista["location"]=  type_point([paralista["longitud"],paralista["latitud"]])
    unjsonnuevo.append(paralista)

In [31]:
unjsonnuevo[0]

{'nombre': 'La Terraza de Las Rozas Village',
 'latitud': 40.518263334929124,
 'longitud': -3.9031637657058136,
 'location': {'type': 'Point',
  'coordinates': [-3.9031637657058136, 40.518263334929124]}}

#### exportamos el diccionaro a un json

In [32]:
# Exportamos el json con json.dump y luego .view_data(json_file="ruta")
import json
with open('prueba.json', 'w') as f:
    json.dump(unjsonnuevo, f)